
# Load nau 

# 20240920
# MODULE/01.4 ★ load_nau_EQPID_추가_240920  를 그대로 카피함. 

# 20240922 : UNIQUE_ID에 M_TIME 구분자 추가
 현재 unique_id : m_time 구분자 없음. 
 문제점 : 동일wafer가 adi, oco 데이터 둘다 존재하면 에러발생.  (M_STEPSEQ 정보가 없음...ㅜ )
 변경점 : unique_id에 m_time 추가함. 


 # 20240922 : M_STEP 정보추가
 현재 : M_STEP 정보가 NAU파일 내부에 없음.
 문제점 : ADI, OCO 섞여있으면 스텝구분을 못해줌..
 변경점 : 파일제목에 있는 M_STEP정보를 끌고 오게 변경 




===================================================================================================================

 # 20241003 
 copy from  "01.5 Load nau 정리_240922.ipynb"
 Oncell residual로부터 Ideal PSM 을 뽑기 위해 제작
 
 1. PSM 전처리 과정 추가 (nau파일로부터 'PerShorMRC"시트에서 정보 끌어옴)



===================================================================================================================
 
 # 20241004 : X_reg, Y_reg 를 뒤쪽으로 이동
 1.  RawData-1 정리할때, X_reg 정보를 맨 뒷쪽으로 이동.   
    - 목적 : X_reg 뒤에 mrc_fit, pred, resi 등등을 붙여서 병합하려고 함

 2. MRC_RX, MRC_RY 도 Vloopup으로 test no에 맞춰서 개별행마다 넣어줌. 
    - 목적 : MRC_X, MRC_RX,  MRC_RX-MRC_X 를 구분해주고자함. 
    MRC_X : -PSM + PointMRC
    MRC_RX : PointMRC
    MRC_X - MRC_RX : -PSM 
    -(MRC_X - MRC_RX) : PSM 

    

# 20241005 
1. 077251의 경우에는 E1,E2,E3 3개의 NAU 파일이 있어서 중복됨.  E2,E3로 끝나는 파일은 제거처리해야함.
2. TEST NO로 E1, E2, E3로 구분지어서 Unique_id에 구분자 넣어줘야함. 
   E1 = TEST1~80 (G2A)
   E2 = TEST81~160 (G2D)
   E3 = TEST161~240 (D2A)

3. PSM Fitting시에 문제발생.    PerShorMRC 시트의 unique id와  RawData-1의 unique id(E1,E2,E3붙음)가 다름.   PSM과 동일한 형식으로 Unique ID3을 추가해봄.

4. GBL에서 DCC설비구분 못해주고 있음.(BASEEQP에는 1.0정보만 있음)
   -> MMO_MRC_Eqp 정보를 끌어오게 추가. 
      RawData-1에 정리할때 신규컬럼으로 추가. unique id에 추가. m3s trend에서 mmo설비별로 색깔 구분해줘야함. 
      
   -> PerShotMRC 시트에서 신규컬럼 추가되어 한줄 밀려남 ->  교정. # MMO_MRC_EQP 신규컬럼 추가되면서 15:87 -> 16:88로 변경함


# 20241006 
1. 엑셀에 저장하면 속도 오래걸림 -> csv 형태로 따로 저장

# 20241008 : M_STEP 불러오려면 시간 오래걸림. SKIP한 버전

# 202041008(2) : shotmap 그릴때 참고하기 위해 chip갯수 정보 추출




In [4]:


import pandas as pd
import numpy as np
import os
from openpyxl import load_workbook
import openpyxl
import time
from datetime import datetime






# nau 파일이 있는 폴더 경로
folder_path = 'C:/py_data/nau'

print(datetime.now().strftime('%Y-%m-%d %H:%M:%S'), '작업 시작')



################################################ E1만 남기고 E2, E3는 삭제  ########################################

# 중복 파일 제거 로직 추가
unique_files = {}


# 폴더 내의 모든 파일에 대해 실행
for file_name in os.listdir(folder_path):
    # 파일 경로 설정
    file_path = os.path.join(folder_path, file_name)

    # 파일인지 확인 (폴더는 제외) 및 .nau 확장자 파일만 처리
    if os.path.isfile(file_path) and file_name.endswith('.nau'):
        # 파일명을 "_" 기준으로 분할
        file_name_without_extension = os.path.splitext(file_name)[0]
        split_file_name = file_name_without_extension.split("_")
        
        # 파일의 기본 이름 및 E1, E2, E3 구분자 추출
        base_name = "_".join(split_file_name[:-1])
        file_suffix = split_file_name[-1]

        # 중복 파일 처리: E1 파일만 남기고 나머지 E2, E3는 제거
        if base_name not in unique_files:
            # 처음 등장하는 파일은 E1 파일인지 확인 후 저장
            if file_suffix == "E1":
                unique_files[base_name] = file_path
        else:
            # E2, E3로 끝나는 파일은 삭제
            if file_suffix in ["E2", "E3"]:
                try:
                    os.remove(file_path)
                    print(f"중복 파일 {file_name} 삭제 완료")
                except Exception as e:
                    print(f"{file_name} 파일 삭제 중 에러 발생: {e}")



''' 
################################################ NAU파일제목에서 M_STEP정보 끌어와서 시트에 저장  ########################################

# 폴더 내의 모든 파일에 대해 실행
for file_name in os.listdir(folder_path):
    # 파일 경로 설정
    file_path = os.path.join(folder_path, file_name)
    
    # 파일인지 확인 (폴더는 제외)
    if os.path.isfile(file_path) and file_name.endswith('.nau'):
        try:
            # 파일 확장자를 임시로 .xlsx로 변경
            temp_file_path = file_path.replace('.nau', '.xlsx')
            os.rename(file_path, temp_file_path)

            # pandas를 이용해 엑셀 파일(유사)을 읽기
            excel_data = pd.read_excel(temp_file_path, engine='openpyxl')
            print(f"{file_name} 파일 읽기 성공")

            # 파일명을 "_" 기준으로 분할
            file_name_without_extension = os.path.splitext(file_name)[0]
            split_file_name = file_name_without_extension.split("_")

            # 기존 엑셀 파일 열기
            workbook = load_workbook(temp_file_path)

            # 파일명 시트를 첫 번째로 추가
            new_sheet_name = "FileName"
            if new_sheet_name not in workbook.sheetnames:
                new_sheet = workbook.create_sheet(title=new_sheet_name, index=0)  # 첫 번째 시트로 추가
            else:
                new_sheet = workbook[new_sheet_name]

            # 분할된 파일명을 첫 번째 시트에 기록
            for col, value in enumerate(split_file_name, start=1):
                new_sheet.cell(row=1, column=col, value=value)

            # 변경된 파일 저장
            workbook.save(temp_file_path)
            print(f"{file_name} 파일에 새로운 시트 추가 완료")

            # 확장자를 다시 .nau로 복원
            os.rename(temp_file_path, file_path)

        except Exception as e:
            print(f"{file_name} 파일을 읽는 중 에러 발생: {e}")

'''


################################################ NAU로부터 필요데이터 추출 (전처리) ########################################




# 추출할 컬럼 위치 설정 (예: 첫 번째 열은 0, 두 번째 열은 1로 인덱스 시작)
columns_to_extract = [0, 1, 2, 3, 4, 5, 6, 7]  # Wafer, TEST, DieX, DieY, X_reg, Y_reg, MRC_X, MRC_Y의 열 위치

def process_nau_files(folder_path, columns_to_extract):
    # 결과를 담을 리스트 생성
    combined_rawdata_list = []
    combined_trocs_input_list = []
    mrc_data_list = []
    combined_psm_input_list = []

    # 폴더 내 모든 nau 파일에 대해 반복
    for file_name in os.listdir(folder_path):
        if file_name.endswith('.nau'):
            file_path = os.path.join(folder_path, file_name)
            
            # 필요한 시트만 읽기
            rawdata_file = pd.read_excel(file_path, sheet_name='RawData-1')
            trocs_input_file = pd.read_excel(file_path, sheet_name='Trocs Input')
            psm_input_file = pd.read_excel(file_path, sheet_name='PerShotMRC')



            ######################################## RawData-1 전처리 ##################################################
            # 지정된 열 추출
            extracted_data_raw = rawdata_file.iloc[:, columns_to_extract].copy()


            ##### test no로 E1,E2,E3 구분하기 ######

            extracted_data_raw['GROUP'] = pd.cut(extracted_data_raw['TEST'], bins=[0, 80, 160, 240], labels=['E1', 'E2', 'E3'])
 

            # 'LOT_ID' 값 및 'STEPSEQ' 추출
            lot_id_value_raw = rawdata_file.columns[13]
            stepseq_value_raw = rawdata_file.iloc[0, 13]
            wafer_value_raw = rawdata_file.iloc[0, 0]
            p_eqpid__value_raw = rawdata_file.iloc[1, 13]
            photo_ppid_value_raw = rawdata_file.iloc[11, 13]
            p_time_value_raw = rawdata_file.iloc[2, 13]
            m_time_value_raw = rawdata_file.iloc[4, 13]
            chuckid_value_raw = rawdata_file.iloc[15, 13]
            mmo_mrc_eqp_value_raw = rawdata_file.iloc[19, 13]


            # 'm_step' 값 추출 및 신규컬럼 추가


                
            # 새로운 컬럼 추가
            extracted_data_raw['STEPSEQ'] = stepseq_value_raw
            extracted_data_raw['LOT_ID'] = lot_id_value_raw

            # 추가 정보 추출 및 컬럼 추가
            extracted_data_raw['STEP_PITCH_X'] = rawdata_file.iloc[6, 13]
            extracted_data_raw['STEP_PITCH_Y'] = rawdata_file.iloc[7, 13]
            extracted_data_raw['MAP_SHIFT_X'] = rawdata_file.iloc[8, 13]
            extracted_data_raw['MAP_SHIFT_Y'] = rawdata_file.iloc[9, 13]



            # 'coordinate_X', 'coordinate_Y' 매핑
            # extracted_data_raw['TEST']에 있는 값들이 coord_map에 있는 'Test No'와 일치하면 coordinate_X와 coordinate_Y 값이 채워집니다.
            coord_map = rawdata_file[['Test No', 'coordinate_X', 'coordinate_Y']].drop_duplicates(subset='Test No').set_index('Test No')
            extracted_data_raw['coordinate_X'] = extracted_data_raw['TEST'].map(coord_map['coordinate_X'])
            extracted_data_raw['coordinate_Y'] = extracted_data_raw['TEST'].map(coord_map['coordinate_Y'])

            ##### 'MRC_RX', 'MRC_RY' 매핑
            coord_map = rawdata_file[['Test No', 'MRC_RX', 'MRC_RY']].drop_duplicates(subset='Test No').set_index('Test No')
            extracted_data_raw['MRC_RX'] = extracted_data_raw['TEST'].map(coord_map['MRC_RX'])
            extracted_data_raw['MRC_RY'] = extracted_data_raw['TEST'].map(coord_map['MRC_RY'])

            ##### MRC_X - MRC_RX 계산해서 PSM만 따로 추출 
            mrc_x_minus_mrc_rx = extracted_data_raw['MRC_X'] - extracted_data_raw['MRC_RX']
            mrc_y_minus_mrc_ry = extracted_data_raw['MRC_Y'] - extracted_data_raw['MRC_RY']
            extracted_data_raw['PSM_X'] = 0 - mrc_x_minus_mrc_rx
            extracted_data_raw['PSM_Y'] = 0 - mrc_y_minus_mrc_ry

                        
                  
            # 'wf_x' 및 'wf_y' 계산
            extracted_data_raw['wf_x'] = (
                extracted_data_raw['DieX'] * extracted_data_raw['STEP_PITCH_X'] +
                extracted_data_raw['MAP_SHIFT_X'] + extracted_data_raw['coordinate_X']
            )
            extracted_data_raw['wf_y'] = (
                extracted_data_raw['DieY'] * extracted_data_raw['STEP_PITCH_Y'] +
                extracted_data_raw['MAP_SHIFT_Y'] + extracted_data_raw['coordinate_Y']
            )

            ##### context 정보추가 #####
            extracted_data_raw['P_EQPID'] = rawdata_file.iloc[1, 13]
            extracted_data_raw['P_TIME'] = rawdata_file.iloc[2, 13]
            extracted_data_raw['M_TIME'] = rawdata_file.iloc[4, 13]
            extracted_data_raw['Photo_PPID'] = rawdata_file.iloc[11, 13]
            extracted_data_raw['Base_EQP1'] = rawdata_file.iloc[12, 13]
            extracted_data_raw['ChuckID'] = rawdata_file.iloc[15, 13]
            extracted_data_raw['ReticleID'] = rawdata_file.iloc[16, 13]
            extracted_data_raw['MMO_MRC_EQP'] = rawdata_file.iloc[19, 13]
            extracted_data_raw['CHIP_X_NUM'] = rawdata_file.iloc[25, 13]
            extracted_data_raw['CHIP_Y_NUM'] = rawdata_file.iloc[26, 13]

            ######## 'Unique_ID'라는 새로운 컬럼 추가   ########  
            ## M_TIME 추가(240922)

            extracted_data_raw['UNIQUE_ID'] = extracted_data_raw.apply(
                lambda row: f"{row['STEPSEQ']}_{row['P_EQPID']}_{row['Photo_PPID']}_{row['MMO_MRC_EQP']}_{row['P_TIME']}_{row['M_TIME']}_{row['LOT_ID']}_{row['Wafer']}_{row['GROUP']}", axis=1)


            # 'Unique_ID2 추가 (TEST, DieX, DieY 추가. 정렬용)
            extracted_data_raw['UNIQUE_ID2'] = extracted_data_raw.apply(
                lambda row: f"{row['STEPSEQ']}_{row['P_EQPID']}_{row['Photo_PPID']}_{row['MMO_MRC_EQP']}_{row['P_TIME']}_{row['M_TIME']}_{row['LOT_ID']}_{row['Wafer']}_{row['TEST']}_{row['DieX']}_{row['DieY']}_{row['GROUP']} ", axis=1)
            
            extracted_data_raw['UNIQUE_ID3'] = extracted_data_raw.apply(
                lambda row: f"{row['STEPSEQ']}_{row['P_EQPID']}_{row['Photo_PPID']}_{row['MMO_MRC_EQP']}_{row['P_TIME']}_{row['M_TIME']}_{row['LOT_ID']}_{row['Wafer']}", axis=1)
            
                       



            # 컬럼 순서 재조정  ( group e1,e2,e3 추가_241005)
            cols_order = [
                'UNIQUE_ID', 'UNIQUE_ID2', 'UNIQUE_ID3',
                'STEPSEQ', 'LOT_ID', 'Wafer', 
                'P_EQPID', 'Photo_PPID', 'P_TIME', 'M_TIME', 'ChuckID', 'ReticleID', 'Base_EQP1', 'MMO_MRC_EQP',
                'TEST', 'GROUP', 'DieX', 'DieY',
                'STEP_PITCH_X', 'STEP_PITCH_Y', 'MAP_SHIFT_X', 'MAP_SHIFT_Y', 'coordinate_X', 'coordinate_Y', 'wf_x', 'wf_y', 'CHIP_X_NUM', 'CHIP_Y_NUM',
                'X_reg', 'Y_reg', 'MRC_X', 'MRC_Y', 'MRC_RX', 'MRC_RY', 'PSM_X', 'PSM_Y'
            ]
            extracted_data_raw = extracted_data_raw[cols_order]

            # 리스트에 추가
            combined_rawdata_list.append(extracted_data_raw)






            ######################################## Trocs Input 전처리 ##################################################

            trocs_input_file['STEPSEQ'] = stepseq_value_raw
            trocs_input_file['LOT_ID'] = lot_id_value_raw
            trocs_input_file['Wafer'] = wafer_value_raw
            trocs_input_file['P_EQPID'] = p_eqpid__value_raw
            trocs_input_file['Photo_PPID'] = photo_ppid_value_raw
            trocs_input_file['P_TIME'] = p_time_value_raw 
            trocs_input_file['M_TIME'] = m_time_value_raw 
            trocs_input_file['ChuckID'] = chuckid_value_raw
            trocs_input_file['MMO_MRC_EQP'] = mmo_mrc_eqp_value_raw

            

         
            # Trocs Input 데이터프레임에 'Unique_ID'라는 새로운 컬럼 추가
            trocs_input_file['UNIQUE_ID'] = trocs_input_file.apply(
                lambda row: f"{row['STEPSEQ']}_{row['P_EQPID']}_{row['Photo_PPID']}_{row['MMO_MRC_EQP']}_{row['P_TIME']}_{row['M_TIME']}_{row['LOT_ID']}_{row['Wafer']}", axis=1)

            # 'Unique_ID2 추가 (DieX(=dCol), DieY(=dRow) 추가. 정렬용)
            trocs_input_file['UNIQUE_ID2'] = trocs_input_file.apply(
                lambda row: f"{row['STEPSEQ']}_{row['P_EQPID']}_{row['Photo_PPID']}_{row['MMO_MRC_EQP']}_{row['P_TIME']}_{row['M_TIME']}_{row['LOT_ID']}_{row['Wafer']}_{row['dCol']}_{row['dRow']}", axis=1)



            
            # 순서 재조정
            # 기존 컬럼 뒤로 밀기 - 순서를 맨 앞으로 추가하는 순서로 재배열
            cols_to_insert = ['UNIQUE_ID', 'UNIQUE_ID2', 'STEPSEQ', 'LOT_ID', 'Wafer', 'P_EQPID', 'Photo_PPID', 'MMO_MRC_EQP', 'P_TIME', 'M_TIME', 'ChuckID']

            # 각 컬럼을 지정된 위치에 삽입 (맨 앞에)
            for i, col in enumerate(cols_to_insert):
                trocs_input_file.insert(i, col, trocs_input_file.pop(col))

            # 리스트에 추가
            combined_trocs_input_list.append(trocs_input_file)





            ######################################## MRC 전처리 ##################################################

            
            # MRC 데이터 추출을 위해 'RawData-1' 시트를 header=None으로 다시 읽음
            rawdata_file_no_header = pd.read_excel(file_path, sheet_name='RawData-1', header=None)

            # MRC 데이터 추출
            mrc_part1 = rawdata_file_no_header.iloc[0:20, 15:17]
            mrc_part2 = rawdata_file_no_header.iloc[22:40, 15:17]
            mrc_part = pd.concat([mrc_part1, mrc_part2], ignore_index=True)

            # 컬럼 이름 설정
            mrc_part.columns = ['K PARA', 'GPM']

            # INDEX 컬럼 추가 (1부터 시작하는 순차적 번호)
            mrc_part['INDEX'] = range(1, len(mrc_part) + 1)


            # 컬럼 추가             
            mrc_part['STEPSEQ'] = stepseq_value_raw
            mrc_part['LOT_ID'] = lot_id_value_raw
            mrc_part['Wafer'] = wafer_value_raw
            mrc_part['P_EQPID'] = p_eqpid__value_raw
            mrc_part['Photo_PPID'] = photo_ppid_value_raw
            mrc_part['P_TIME'] = p_time_value_raw 
            mrc_part['M_TIME'] = m_time_value_raw 
            mrc_part['ChuckID'] = chuckid_value_raw
            mrc_part['MMO_MRC_EQP'] = mmo_mrc_eqp_value_raw




            # mrc_part 데이터프레임에 'Unique_ID'라는 새로운 컬럼 추가
            mrc_part['UNIQUE_ID'] = mrc_part.apply(
                lambda row: f"{row['STEPSEQ']}_{row['P_EQPID']}_{row['Photo_PPID']}_{row['MMO_MRC_EQP']}_{row['P_TIME']}_{row['M_TIME']}_{row['LOT_ID']}_{row['Wafer']}", axis=1)


            # 컬럼 순서 재조정
            mrc_cols_order = [
                'UNIQUE_ID',
                'STEPSEQ', 'LOT_ID', 'Wafer', 
                'P_EQPID', 'Photo_PPID', 'MMO_MRC_EQP','P_TIME', 'M_TIME', 'ChuckID',  
                'K PARA', 'GPM', 'INDEX'
            ]
            mrc_part = mrc_part[mrc_cols_order]

            
            # 리스트에 추가
            mrc_data_list.append(mrc_part)



            
            ######################################## PSM 전처리 ##################################################

            psm_input_file['STEPSEQ'] = stepseq_value_raw
            psm_input_file['LOT_ID'] = lot_id_value_raw
            psm_input_file['Wafer'] = wafer_value_raw
            psm_input_file['P_EQPID'] = p_eqpid__value_raw
            psm_input_file['Photo_PPID'] = photo_ppid_value_raw
            psm_input_file['P_TIME'] = p_time_value_raw 
            psm_input_file['M_TIME'] = m_time_value_raw 
            psm_input_file['ChuckID'] = chuckid_value_raw
            psm_input_file['MMO_MRC_EQP'] = mmo_mrc_eqp_value_raw


            # PSM Input 데이터프레임에 'Unique_ID'라는 새로운 컬럼 추가
            psm_input_file['UNIQUE_ID'] = psm_input_file.apply(
                lambda row: f"{row['STEPSEQ']}_{row['P_EQPID']}_{row['Photo_PPID']}_{row['MMO_MRC_EQP']}_{row['P_TIME']}_{row['M_TIME']}_{row['LOT_ID']}_{row['Wafer']}", axis=1)

            # 'Unique_ID2 추가 (DieX(=dCol), DieY(=dRow) 추가. 정렬용)
            psm_input_file['UNIQUE_ID2'] = psm_input_file.apply(
                lambda row: f"{row['STEPSEQ']}_{row['P_EQPID']}_{row['Photo_PPID']}_{row['MMO_MRC_EQP']}_{row['P_TIME']}_{row['M_TIME']}_{row['LOT_ID']}_{row['Wafer']}_{row['dCol']}_{row['dRow']}", axis=1)
            
            
            # 순서 재조정
            # 기존 컬럼 뒤로 밀기 - 순서를 맨 앞으로 추가하는 순서로 재배열
            cols_to_insert_psm = ['UNIQUE_ID', 'UNIQUE_ID2', 'STEPSEQ', 'LOT_ID', 'Wafer', 'P_EQPID', 'Photo_PPID', 'MMO_MRC_EQP', 'P_TIME', 'M_TIME', 'ChuckID']

            # 각 컬럼을 지정된 위치에 삽입 (맨 앞에)
            for i, col in enumerate(cols_to_insert_psm):
                psm_input_file.insert(i, col, psm_input_file.pop(col))

            # 리스트에 추가
            combined_psm_input_list.append(psm_input_file)






    # 리스트를 데이터프레임으로 병합
    combined_rawdata = pd.concat(combined_rawdata_list, ignore_index=True)
    combined_trocs_input = pd.concat(combined_trocs_input_list, ignore_index=True)
    mrc_data = pd.concat(mrc_data_list, ignore_index=True)
    combined_psm_input = pd.concat(combined_psm_input_list, ignore_index=True)




    # 병합 후, UNIQUE_ID기준으로 오름차순 정렬 (내림차순하고 싶으면 ascending=False 옵션 추가하면 됨.) 
    combined_rawdata = combined_rawdata.sort_values(by=['UNIQUE_ID', 'TEST', 'DieX', 'DieY'])    
    combined_trocs_input = combined_trocs_input.sort_values(by=['UNIQUE_ID', 'dCol', 'dRow'])
    mrc_data = mrc_data.sort_values(by=['UNIQUE_ID', 'INDEX'])
    combined_psm_input = combined_psm_input.sort_values(by=['UNIQUE_ID', 'dCol', 'dRow'])


    ''' 
    # 최종 데이터를 엑셀 파일로 저장
    with pd.ExcelWriter('output.xlsx') as writer:
        combined_rawdata.to_excel(writer, sheet_name='RawData-1', index=False)
        combined_trocs_input.to_excel(writer, sheet_name='Trocs Input', index=False)
        mrc_data.to_excel(writer, sheet_name='MRC', index=False)
        combined_psm_input.to_excel(writer, sheet_name='PerShotMRC', index=False)
    '''

    # 최종 데이터를 CSV 파일로 저장
    combined_rawdata.to_csv('RawData-1.csv', index=False)
    combined_trocs_input.to_csv('Trocs_Input.csv', index=False)
    mrc_data.to_csv('MRC.csv', index=False)
    combined_psm_input.to_csv('PerShotMRC.csv', index=False)

























################################### nau 파일 처리 및 데이터 저장 #####################################################################

print(datetime.now().strftime('%Y-%m-%d %H:%M:%S'), 'nau파일 전처리 시작.')
process_nau_files(folder_path, columns_to_extract)
print(datetime.now().strftime('%Y-%m-%d %H:%M:%S'), 'nau파일 전처리 완료.')









2024-10-08 00:37:05 작업 시작
2024-10-08 00:37:05 nau파일 전처리 시작.
2024-10-08 00:37:07 nau파일 전처리 완료.
